## __SVM, 서포트 벡터 머신__
---

최적의 결정경계인 <b>초평면</b>을 찾아 분류/회귀를 하는 지도학습 기법입니다.  
복잡한 분류와, 소/중 규모의 데이터 세트에 적합합니다.  

* `sklearn의 .svm.SVC, .SGDClassifier(loss='hinge', alpha=1/(m*C))`  
 SVC가 더 빠르지만, SGD가 메모리 효율적이기 때문에 큰 데이터세트나 한정된 컴퓨팅 성능에 유리합니다.  

#### __0. 용어__
초평면: SVM이 찾고자 하는 목표입니다. 새로운 데이터를 분류/예측하는 기준이 됩니다.  
서포트 벡터: 초평면을 결정하는 각 클래스의 벡터(데이터)입니다.   
마진: 초평면과 서포트 벡터간의 간격입니다.  
마진 오류: 마진 사이에 있는 샘플로, 초평면에 의해 잘못 분류될 수 있는 것들입니다.  
> 마진 간격이 넓은 것이 바람직한 모델로 스케일링의 유무가 중요하게 작동합니다.  

### __1. 소프트마진 분류__
<b>하드마진 분류</b>란 초평면으로 완벽하게 구분되고, 마진오류 또한 없는 경우를 말합니다.  
하지만 데이터에 너무 의존적이며 이상치 하나에도 영향을 많이 받습니다.  

<b>소프트마진 분류</b>는 마진오류를 감수하면서 마진의 폭을 적절히 넓혀 일반화를 더 중시하는 방법입니다.  
규제를 가하면 마진이 넓어져 이를 구현할 수 있습니다.  

### __2. 비선형 분류__
4장의 다항회귀와 마찬가지로 데이터의 다항식 값을 추가하여 비선형 데이터를 처리할 수 있습니다.  
방식은 동일하지만, SVM에서는 많은 계산량을 보완해줄 방법이 있습니다.  

#### __2.0 커널트릭__
이 방법은 비선형 분류를 위한 실제 특성을 추가하지 않으면서 보다 적은 연산량으로 동일한 효과를 내는 기법입니다.  
다양한 커널이 있지만, 비선형 분류에서는 다항식 커널, RBF 커널을 주로 사용합니다.  

* SVC의 커널을 linear로 주어 선형 SVM을 시행할 수 있지만, linearSVC가 최적화되어있어 훨씬 빠릅니다.

#### __2.1 다항식 커널__
다항식 값을 이용하여 계산하는 커널입니다.  
#### __2.2 RBF 커널__
각 샘플이 특정 <b>랜드마크 ($\ell$)</b>와 얼마나 유사한지 측정하는 <b>유사도 함수</b>의 값을 계산하는 커널입니다.  
유사도함수 RBf는 (Radial Basis Function, 방사기저함수) 아래와 같습니다. 
> $\phi_{\gamma}(\mathbf{x}, \ell)=exp(-\gamma \lVert \mathbf{x}-\ell \rVert ^2)$

이 함수는 0~1값을 가지는 가우시안 함수로, 랜드마크와 같은 위치의 경우 1입니다.  
감마가 높아지면 종 모양이 좁아지며 각 샘플에 더 민감하게 변화하기 때문에 낮은 감마를 규제와 비슷하게 사용할 수 있습니다.  

랜드마크를 여러개 지정하여 각 샘플을 각 랜드마크의 RBF로 변환, 고차원에서 선형적 구분을 시도합니다.  
일반적으로 모든 샘플을 랜드마크로 설정하여 샘플이 너무 많은 경우 연산량이 매우 많아질 수 있습니다.  